**6820017_ArimitraMaiti_IA1**

### <center>Tripadvisor Review Multinomial Classification in Jupyter (Part-I)</center>
<center>Big Data Analytics - Analysis of Text and Social Media Data: Individual Assignment Week-3</center>
<center>ePGD ABA 2020-21</center>

**First notebook for data collection from Tripadvisor xpaths**

**CAUTION - This notebook takes roughly 5 hours to complete execution (excluding model train) \
This notebook may not run end to end in google colab environment or mac environment \
The input file for this notebook comes from the individual websites collected from tripadvisor**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy.stats as stats
import math
import sys
import string
import re
import requests
from lxml import html

In [2]:
# !pip install selenium

In [3]:
import csv
import time
from selenium import webdriver

In [4]:
##Function to extract scores, date, title, review comments from one particular url
##On an average 9 to 10 records should come from one single run of this function
def fetch_single_link(URL):
    ##It is critical to point the chorme driver location while running this function
    ##Link to download compatible version is https://sites.google.com/a/chromium.org/chromedriver/downloads
    driver = webdriver.Chrome(executable_path='C:/Users/sinaii/Documents/Python Scripts/NLP/chromedriver.exe')
    driver.get(URL)
    
    # Find and click the More link (to load all reviews)
    driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
    time.sleep(5) # Wait for reviews to load
    
    reviews = driver.find_elements_by_xpath("//div[@class='ui_column is-9']")
    num_page_items = min(len(reviews), 10)
    
    store = []
    # Loop through the reviews found
    for i in range(num_page_items):
        # get the score, date, title and review
        score_class = reviews[i].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
        score = score_class.split("_")[3]
        date = reviews[i].find_element_by_xpath(".//span[@class='ratingDate']").get_attribute("title")
        title = reviews[i].find_element_by_xpath(".//span[@class='noQuotes']").text
        review = reviews[i].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", "")
        mylist = [score, date, title, review]
        store.append(mylist)
    
    driver.close()
    return store

In [5]:
##Import urls for tripadvisor links
input_url = "https://raw.githubusercontent.com/arimitramaiti/datasets/master/articles/urls_ia1.csv"
dataset = pd.read_csv(input_url, error_bad_lines = False, header=0, index_col=None)
dataset.head(4)

,Link_ID,Link_address,Name,Location,City
0,1,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,Panaji,Goa
1,2,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,Panaji,Goa
2,3,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,Panaji,Goa
3,4,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,Panaji,Goa


In [6]:
iter_ = dataset['Link_address']

In [6]:
###Use the above function to iterate for all 401 urls imported from cloud
##This function may stop after extracting 500-800 links due to chrome driver restart
##Therefore all 401 links have been collated on three runs os this notebook
result = pd.DataFrame(columns=['score', 'date', 'title', 'review', 'link_id'])
for k in iter_:
    res = fetch_single_link(k)
    res = pd.DataFrame(res, columns=['score', 'date', 'title', 'review'])
    res['link_id'] = k
    result = pd.concat([result, res], ignore_index=True, axis=0)

In [7]:
result.shape

(611, 5)

In [8]:
##Dictionary to map urls to reviews
df = input_url[['Link_address', 'Name', 'Location', 'City']]

dfT = df.set_index(keys='Link_address').T

df_dict = dfT.to_dict("list")

In [9]:
result1 = result.copy()

In [10]:
result1['Values'] = result['link_id'].map(df_dict)

In [11]:
result1['Values'] = result1.Values.astype('str')

result1['Name'] = result1.Values.str.split(',').str[0].apply(lambda x: str(x.replace('[','')))

result1['Location'] = result1.Values.str.split(',').str[1]

result1['City'] = result1.Values.str.split(',').str[2].apply(lambda x: str(x.replace(']','')))

result1['Name'] = result1.Name.apply(lambda x: str(x.replace("'","")))
result1['Location'] = result1.Location.apply(lambda x: str(x.replace("'","")))
result1['City'] = result1.City.apply(lambda x: str(x.replace("'","")))

In [12]:
result1.drop(labels=['Values'], axis=1, inplace=True)

In [13]:
result1.head()

,score,date,title,review,link_id,Name,Location,City
0,50,29 November 2015,Best Chinese restaurant!!,Mainland China at sector 18 Noida is the heave...,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,K1 Plot,Noida
1,20,28 November 2015,No-no,If you are an exclusive Chinese food lover tha...,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,K1 Plot,Noida
2,40,27 November 2015,One Of The Best Chinese Restaurant,Mainland China for me is ever green. I have al...,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,K1 Plot,Noida
3,40,26 November 2015,Nice chinese buffet,Mainland China offers pretty nice chinese buff...,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,K1 Plot,Noida
4,40,17 November 2015,Good Restaurant,It is one of the good restaurant in noida beca...,https://www.tripadvisor.in/Restaurant_Review-g...,Mainland China,K1 Plot,Noida


In [14]:
result1.to_csv("result1.csv")

In [15]:
del result1, dfT, df, df_dict

In [40]:
# df1 = pd.read_csv("result1_2250_1.csv", parse_dates=False)

# df2 = pd.read_csv("result1_1130_2.csv", parse_dates=False)

# df3 = pd.read_csv("result1_655_3.csv", parse_dates=False)

# df_final = pd.concat([df1, df2, df3], axis=0)

# df_final.reset_index(drop=True, inplace=True)

# df_final = df_final[['score', 'date', 'title', 'review', 'link_id', 'Name', "Location", "City"]]

# df_final.shape

In [39]:
# df_final.head()

In [38]:
# df_final.to_csv("reviews_mainland_china.csv")

**The output of this notebook is used in Part-II**